In [ ]:
import numpy as np  # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)
import random
import os
import numpy as np  # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F
import numpy as np
import pickle
import torch
from torch.utils.data import Dataset, DataLoader



In [319]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Executing on " + ("cuda:0" if torch.cuda.is_available() else "cpu"))

Executing on cpu


In [320]:
df_train = pd.read_csv("Data/hin/hin_train.csv")
df_test = pd.read_csv('Data/hin/hin_test.csv')
df_valid = pd.read_csv('Data/hin/hin_valid.csv')

In [321]:
PAD_CHAR = '_'
EOW_CHAR = '|'
SOW_CHAR = '$'
BATCH_SIZE = 1

In [322]:
eng_alphabets = [chr(alpha) for alpha in range(ord('a'), ord('z') + 1)]
# eng_alpha2index = {pad_char:0}
in_dict = {PAD_CHAR: 0, EOW_CHAR: 1, SOW_CHAR: 2}
for index, alpha in enumerate(eng_alphabets):
	in_dict[alpha] = index + 3
print(in_dict)


{'_': 0, '|': 1, '$': 2, 'a': 3, 'b': 4, 'c': 5, 'd': 6, 'e': 7, 'f': 8, 'g': 9, 'h': 10, 'i': 11, 'j': 12, 'k': 13, 'l': 14, 'm': 15, 'n': 16, 'o': 17, 'p': 18, 'q': 19, 'r': 20, 's': 21, 't': 22, 'u': 23, 'v': 24, 'w': 25, 'x': 26, 'y': 27, 'z': 28}


In [323]:
hindi_alphabets = [chr(alpha) for alpha in range(2304, 2432)]
hindi_alphabet_size = len(hindi_alphabets)
out_dict = {PAD_CHAR: 0, EOW_CHAR: 1, SOW_CHAR: 2}
for index, alpha in enumerate(hindi_alphabets):
	out_dict[alpha] = index + 3

In [324]:
print(out_dict)
print(len(out_dict))

{'_': 0, '|': 1, '$': 2, 'ऀ': 3, 'ँ': 4, 'ं': 5, 'ः': 6, 'ऄ': 7, 'अ': 8, 'आ': 9, 'इ': 10, 'ई': 11, 'उ': 12, 'ऊ': 13, 'ऋ': 14, 'ऌ': 15, 'ऍ': 16, 'ऎ': 17, 'ए': 18, 'ऐ': 19, 'ऑ': 20, 'ऒ': 21, 'ओ': 22, 'औ': 23, 'क': 24, 'ख': 25, 'ग': 26, 'घ': 27, 'ङ': 28, 'च': 29, 'छ': 30, 'ज': 31, 'झ': 32, 'ञ': 33, 'ट': 34, 'ठ': 35, 'ड': 36, 'ढ': 37, 'ण': 38, 'त': 39, 'थ': 40, 'द': 41, 'ध': 42, 'न': 43, 'ऩ': 44, 'प': 45, 'फ': 46, 'ब': 47, 'भ': 48, 'म': 49, 'य': 50, 'र': 51, 'ऱ': 52, 'ल': 53, 'ळ': 54, 'ऴ': 55, 'व': 56, 'श': 57, 'ष': 58, 'स': 59, 'ह': 60, 'ऺ': 61, 'ऻ': 62, '़': 63, 'ऽ': 64, 'ा': 65, 'ि': 66, 'ी': 67, 'ु': 68, 'ू': 69, 'ृ': 70, 'ॄ': 71, 'ॅ': 72, 'ॆ': 73, 'े': 74, 'ै': 75, 'ॉ': 76, 'ॊ': 77, 'ो': 78, 'ौ': 79, '्': 80, 'ॎ': 81, 'ॏ': 82, 'ॐ': 83, '॑': 84, '॒': 85, '॓': 86, '॔': 87, 'ॕ': 88, 'ॖ': 89, 'ॗ': 90, 'क़': 91, 'ख़': 92, 'ग़': 93, 'ज़': 94, 'ड़': 95, 'ढ़': 96, 'फ़': 97, 'य़': 98, 'ॠ': 99, 'ॡ': 100, 'ॢ': 101, 'ॣ': 102, '।': 103, '॥': 104, '०': 105, '१': 106, '२': 107, '३': 108, '४': 109, '५': 110,

In [325]:
if df_train.iloc[0][0][0] != SOW_CHAR:
	df_train = df_train.iloc[:, ].apply(lambda x: SOW_CHAR + x + EOW_CHAR)
	df_test = df_test.iloc[:, ].apply(lambda x: SOW_CHAR + x + EOW_CHAR)
	df_valid = df_valid.iloc[:, ].apply(lambda x: SOW_CHAR + x + EOW_CHAR)


In [326]:
df_train = df_train.set_axis(['X', 'Y'], axis=1)
df_valid = df_valid.set_axis(['X', 'Y'], axis=1)
df_test = df_test.set_axis(['X', 'Y'], axis=1)

In [327]:
print(df_train)
print(df_test)
print(df_valid)


                   X            Y
0          $bindhya|   $बिन्द्या|
1        $kirankant|   $किरणकांत|
2      $yagyopaveet|  $यज्ञोपवीत|
3          $ratania|    $रटानिया|
4       $vaganyache|  $वागण्याचे|
...              ...          ...
51194        $toned|       $टोंड|
51195   $mutanaazaa|   $मुतनाज़ा|
51196    $asahmaton|    $असहमतों|
51197    $sulgaayin|   $सुलगायीं|
51198  $anchuthengu|  $अंचुतेंगु|

[51199 rows x 2 columns]
                   X              Y
0        $sikhaaega|      $सिखाएगा|
1            $learn|         $लर्न|
2         $twitters|     $ट्विटर्स|
3      $tirunelveli|  $तिरुनेलवेली|
4     $independence|  $इंडिपेंडेंस|
...              ...            ...
4090       $saflata|       $सफ़लता|
4091        $shbana|        $शबाना|
4092  $khaatootolaa|     $खातूटोला|
4093    $shivastava|     $शिवास्तव|
4094  $preranapuree|  $प्रेरणापुरी|

[4095 rows x 2 columns]
                  X              Y
0           $bajai|         $बजाई|
1       $sanghthan|        $संघठन|
2   

In [328]:
max_input_length = max(df_train.iloc[:, 0].apply(lambda x: len(x)).max(),
					   df_test.iloc[:, 0].apply(lambda x: len(x)).max(),
					   df_valid.iloc[:, 0].apply(lambda x: len(x)).max())

max_output_length = max(df_train.iloc[:, 1].apply(lambda x: len(x)).max(),
						df_test.iloc[:, 1].apply(lambda x: len(x)).max(),
						df_valid.iloc[:, 1].apply(lambda x: len(x)).max())

print("max input length", max_input_length)
print("max output length", max_output_length)
MAX_LENGTH = max(max_input_length, max_output_length)
print("max_length", MAX_LENGTH)

max input length 28
max output length 22
max_length 28


In [329]:
input_vocab_size = len(in_dict)
output_vocab_size = len(out_dict)
print("Input Character max", input_vocab_size)
print("output Character size", output_vocab_size)

train = df_train.values.tolist()
valid = df_valid.values.tolist()
test = df_test.values.tolist()


Input Character max 29
output Character size 131


In [330]:
input_vocab_size = len(in_dict)
output_vocab_size = len(out_dict)
print("Input Character max", input_vocab_size)
print("output Character size", output_vocab_size)

train = df_train.values.tolist()
valid = df_valid.values.tolist()
test = df_test.values.tolist()

Input Character max 29
output Character size 131


In [331]:
def inputToTensor(line):
	print([in_dict[x] for x in line])
	tensor = torch.tensor(data=([in_dict[x] for x in line]), dtype=torch.long)
	return tensor


def charToTensor(char, dic=in_dict):
	tensor = torch.zeros(len(dic))
	tensor[dic[char]] = 1
	return tensor


def outToTensor(word):
	tensor = torch.tensor([out_dict[x] for x in word])
	return tensor


In [332]:
print(inputToTensor(train[0][0]))

[2, 4, 11, 16, 6, 10, 27, 3, 1]
tensor([ 2,  4, 11, 16,  6, 10, 27,  3,  1])


In [333]:
print(train[1][1])

$किरणकांत|


In [334]:
inputToTensor("$bindhya|")

[2, 4, 11, 16, 6, 10, 27, 3, 1]


tensor([ 2,  4, 11, 16,  6, 10, 27,  3,  1])

In [335]:
print(in_dict)

{'_': 0, '|': 1, '$': 2, 'a': 3, 'b': 4, 'c': 5, 'd': 6, 'e': 7, 'f': 8, 'g': 9, 'h': 10, 'i': 11, 'j': 12, 'k': 13, 'l': 14, 'm': 15, 'n': 16, 'o': 17, 'p': 18, 'q': 19, 'r': 20, 's': 21, 't': 22, 'u': 23, 'v': 24, 'w': 25, 'x': 26, 'y': 27, 'z': 28}


In [336]:
print(inputToTensor("hello"))


[10, 7, 14, 14, 17]
tensor([10,  7, 14, 14, 17])


In [337]:
def generate_batch(data_batch):
	#     print(data_batch)
	tensor_data = [inputToTensor(x[0].ljust(max_input_length, PAD_CHAR)) for x in data_batch]
	tensor_target = [outToTensor(x[1].ljust(max_output_length, PAD_CHAR)) for x in data_batch]
	padded_input_batch = torch.nn.utils.rnn.pad_sequence(tensor_data, batch_first=True, padding_value=1)
	padded_output_batch = torch.nn.utils.rnn.pad_sequence(tensor_target, batch_first=True, padding_value=1)
	#     print(tensor_data)
	#     print(padded_input_batch.shape)
	#     print(padded_output_batch.shape)
	return padded_input_batch, padded_output_batch

In [338]:
train_dataloader = DataLoader(train, batch_size=BATCH_SIZE, shuffle=True, collate_fn=generate_batch)
test_dataloader = DataLoader(test, batch_size=BATCH_SIZE, shuffle=False, collate_fn=generate_batch)
valid_dataloader = DataLoader(test, batch_size=BATCH_SIZE, shuffle=False, collate_fn=generate_batch, )


In [339]:
for data,target in train_dataloader:
    print(data)
    print(target)
    print(data.shape)
    print(target.shape)
    if True:
        break

[2, 13, 23, 12, 3, 15, 4, 10, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
tensor([[ 2, 13, 23, 12,  3, 15,  4, 10,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0]])
tensor([[ 2, 24, 68, 31,  5, 48,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0]])
torch.Size([1, 28])
torch.Size([1, 22])


In [340]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()

        self.hid_dim = hid_dim
        self.n_layers = n_layers

        self.embedding = nn.Embedding(input_dim, emb_dim)

        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)

        self.dropout = nn.Dropout(dropout)

    def forward(self, src):
        #src = [src len, batch size]
        print("encoder forward prop")

        embedded = self.dropout(self.embedding(src))
        embedded = embedded.permute(1,0,2)
        print("encoder embedded shape", embedded.shape)
        #embedded = [src len, batch size, emb dim]
        outputs, (hidden, cell) = self.rnn(embedded)
        print("encoder hidden dimension", hidden.shape)
        print("encoder output dimension", outputs.shape)
        print("encoder cell dimension", cell.shape)
        #outputs = [src len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        #outputs are always from the top hidden layer
        return hidden, cell

In [341]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()

        self.output_dim = output_dim
        self.hid_dim = hid_dim
        self.n_layers = n_layers

        self.embedding = nn.Embedding(output_dim, emb_dim)

        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)

        self.fc_out = nn.Linear(hid_dim, output_dim)

        self.dropout = nn.Dropout(dropout)

    def forward(self, input, hidden, cell):

        #input = [batch size]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]

        #n directions in the decoder will both always be 1, therefore:
        #hidden = [n layers, batch size, hid dim]
        #context = [n layers, batch size, hid dim]
        print("decoder hidden shape", hidden.shape)
        print("decoder context shape", cell.shape)
        input = input.unsqueeze(0)

        #input = [1, batch size]

        embedded = self.dropout(self.embedding(input))
        print("decoder embedded shape", embedded.shape)
        #embedded = [1, batch size, emb dim]

        output, (hidden, cell) = self.rnn(embedded, (hidden, cell))
        print("decoder output shape", output.shape)
        print("decoder hidden shape", hidden.shape)
        print("decoder context shape", cell.shape)
        #output = [seq len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]

        #seq len and n directions will always be 1 in the decoder, therefore:
        #output = [1, batch size, hid dim]
        #hidden = [n layers, batch size, hid dim]
        #cell = [n layers, batch size, hid dim]

        prediction = self.fc_out(output.squeeze(0))

        #prediction = [batch size, output dim]
        print("decoder prediction shape", prediction.shape)
        return prediction, hidden, cell

In [342]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()

        self.encoder = encoder
        self.decoder = decoder
        self.device = device

        assert encoder.hid_dim == decoder.hid_dim, \
            "Hidden dimensions of encoder and decoder must be equal!"
        assert encoder.n_layers == decoder.n_layers, \
            "Encoder and decoder must have equal number of layers!"

    def forward(self, src, trg, teacher_forcing_ratio = 0.5):

        #src = [src len, batch size]
        #trg = [trg len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use ground-truth inputs 75% of the time

        batch_size = trg.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim

        #tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)

        #last hidden state of the encoder is used as the initial hidden state of the decoder
        hidden, cell = self.encoder(src)
        print("seq2seq hidden shape", hidden.shape)
        print("seq2seq cell shape", cell.shape)
        #first input to the decoder is the <sos> tokens
        input = trg[0,:]

        for t in range(1, trg_len):
			            #insert input token embedding, previous hidden and previous cell states
            #receive output tensor (predictions) and new hidden and cell states
            output, hidden, cell = self.decoder(input, hidden, cell)

            #place predictions in a tensor holding predictions for each token
            outputs[t] = output

            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio

            #get the highest predicted token from our predictions
            top1 = output.argmax(1)

            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = trg[t] if teacher_force else top1

        return outputs

In [343]:
INPUT_DIM = input_vocab_size
OUTPUT_DIM = output_vocab_size
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512
N_LAYERS = 2
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

In [344]:
enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS, DEC_DROPOUT)

In [345]:
model = Seq2Seq(enc, dec, device).to(device)

In [346]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)

model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(29, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(131, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (fc_out): Linear(in_features=512, out_features=131, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [347]:
optimizer = optim.Adam(model.parameters())

In [348]:
# TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]

criterion = nn.CrossEntropyLoss()

In [349]:
def train(model, iterator, optimizer, criterion, clip):

    model.train()

    epoch_loss = 0

    for i, (data, target) in enumerate(iterator):

        src = data
        trg = target

        optimizer.zero_grad()

        output = model(src, trg)

        #trg = [trg len, batch size]
        #output = [trg len, batch size, output dim]

        output_dim = output.shape[-1]

        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)

        #trg = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]

        loss = criterion(output, trg)

        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)

        optimizer.step()

        epoch_loss += loss.item()

    return epoch_loss / len(iterator)

N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    # start_time = time.time()

    train_loss = train(model, train_dataloader, optimizer, criterion, CLIP)
    # valid_loss = evaluate(model, valid_iterator, criterion)

In [350]:
N_EPOCHS = 1
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    # start_time = time.time()
    train_loss = train(model, train_dataloader, optimizer, criterion, CLIP)
    # valid_loss = evaluate(model, valid_iterator, criterion)

[2, 24, 11, 21, 10, 14, 3, 27, 21, 10, 11, 22, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
encoder forward prop
encoder embedded shape torch.Size([28, 1, 256])
encoder hidden dimension torch.Size([2, 1, 512])
encoder output dimension torch.Size([28, 1, 512])
encoder cell dimension torch.Size([2, 1, 512])
seq2seq hidden shape torch.Size([2, 1, 512])
seq2seq cell shape torch.Size([2, 1, 512])


RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn